In [1]:
# Gerekli kütüphanelerin import edilmesi
import os
import glob
import json
import numpy as np
import random
from tqdm import tqdm
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding,
    Conv1D,
    MaxPooling1D,
    GlobalMaxPooling1D,
    Dense,
    Dropout
)
from tensorflow.keras.optimizers import legacy as legacy_optimizers  # Legacy optimizer'ları kullanıyoruz
import matplotlib.pyplot as plt
from tensorflow.keras.utils import Sequence
import gc

# =======================
# Parametrelerin Tanımlanması
# =======================

# Parametreleri JSON formatında bir sözlük olarak tanımlıyoruz
params = {
    "data": {
        "train_dir": ".",  # Eğitim verilerinin dizini
        "test_dir": ".",    # Test verilerinin dizini
        "sample_ratio": 0.2,     # Örnekleme oranı (0 ile 1 arasında)
        "max_sequence_length": 500,  # Padding yapılacak maksimum sekans uzunluğu
        "num_words": 10000      # Tokenizer'da kullanılacak maksimum kelime sayısı
    },
    "model": {
        "embedding": {
            "output_dim": [32, 128, 256]  # Embedding çıkış boyutları
        },
        "conv1d": {
            "num_layers": [1, 2, 3],  # Conv1D katman sayısı
            "filters": [64, 128, 256],  # Filtre sayıları
            "kernel_sizes": [3, 6]  # Kernel boyutları
        },
        "dense": {
            "num_layers": [1, 2],  # Dense katman sayısı
            "units": [64, 256],  # Dense katman nöron sayıları
            "activations": ["relu"],  # Aktivasyon fonksiyonları
            "dropout_rates": [0.25]  # Dropout oranları
        },
        "optimizer": ["adam"],  # Optimizer seçenekleri
        "learning_rates": [1e-4, 1e-2],  # Öğrenme oranları
        "batch_sizes": [32]  # Batch boyutları
    },
    "training": {
        "max_epochs": 20,      # Maksimum epoch sayısı
        "patience": 3,         # Erken durdurma için sabır değeri
        "max_trials": 400      # Hiperparametre araması için maksimum deneme sayısı
    }
}

# =======================
# Veri Yükleme ve Ön İşleme
# =======================

def load_files(directory, file_extension, sample_ratio=0.2):
    """
    Belirli bir uzantıya sahip dosyaları bir dizinden ve alt dizinlerinden yükler.

    Args:
        directory (str): Aranacak dizin.
        file_extension (str): Dosya uzantısı (örneğin, '.html', '.txt').
        sample_ratio (float): Örnekleme oranı (0 ile 1 arasında).

    Returns:
        List[str]: Dosya içeriklerinin listesi.
    """
    file_contents = []
    pattern = os.path.join(directory, f'**/*{file_extension}')
    files = glob.glob(pattern, recursive=True)

    if not files:
        print(f"{directory} dizininde {file_extension} uzantılı dosya bulunamadı.")
        return []

    if not (0 < sample_ratio <= 1):
        print(f"Geçersiz sample_ratio değeri: {sample_ratio}. 0 ile 1 arasında olmalıdır.")
        sample_ratio = 1.0

    num_samples = max(1, int(len(files) * sample_ratio))
    sampled_files = random.sample(files, min(num_samples, len(files)))

    for filepath in tqdm(sampled_files, desc=f"{directory} dizininden {file_extension} dosyaları yükleniyor"):
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                file_contents.append(file.read())
        except UnicodeDecodeError:
            try:
                with open(filepath, 'r', encoding='latin1') as file:
                    file_contents.append(file.read())
            except Exception as e:
                print(f"{filepath} dosyası okunurken hata oluştu: {e}")
    return file_contents

def load_data(params):
    """
    Eğitim ve test verilerini belirtilen dizinlerden yükler.

    Args:
        params (dict): Parametreleri içeren sözlük.

    Returns:
        Tuple: (train_texts, train_labels, test_texts, test_labels)
    """
    train_dir = params["data"]["train_dir"]
    test_dir = params["data"]["test_dir"]
    sample_ratio = params["data"]["sample_ratio"]

    print("Eğitim verilerini yüklüyor...")
    phishing_train = load_files(os.path.join(train_dir, 'phishing'), '.html', sample_ratio)
    legal_train = load_files(os.path.join(train_dir, 'legal'), '.txt', sample_ratio)

    print("Test verilerini yüklüyor...")
    phishing_test = load_files(os.path.join(test_dir, 'phishingTest'), '.html', sample_ratio)
    legal_test = load_files(os.path.join(test_dir, 'legalTest'), '.html', sample_ratio)

    # Verileri ve etiketleri birleştirme
    train_texts = phishing_train + legal_train
    train_labels = [1] * len(phishing_train) + [0] * len(legal_train)

    test_texts = phishing_test + legal_test
    test_labels = [1] * len(phishing_test) + [0] * len(legal_test)

    return train_texts, train_labels, test_texts, test_labels

# =======================
# Data Generator Sınıfı
# =======================

class DataGenerator(Sequence):
    def __init__(self, texts, labels, tokenizer, max_sequence_length, batch_size, shuffle=True):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_sequence_length = max_sequence_length
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.texts))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.texts) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_texts = [self.texts[i] for i in batch_indexes]
        batch_labels = [self.labels[i] for i in batch_indexes]
        sequences = self.tokenizer.texts_to_sequences(batch_texts)
        padded_sequences = pad_sequences(sequences, maxlen=self.max_sequence_length, padding='post', truncating='post')
        return np.array(padded_sequences), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# =======================
# Model Oluşturma Fonksiyonu
# =======================

def build_model(hp):
    """
    Hiperparametre optimizasyonu için model oluşturma fonksiyonu.

    Args:
        hp: Hiperparametreler.

    Returns:
        tf.keras.Model: Oluşturulan model.
    """
    model = Sequential()
    # Embedding katmanı
    embedding_output_dim = hp.Choice('embedding_output_dim', params["model"]["embedding"]["output_dim"])
    model.add(Embedding(
        input_dim=params["data"]["num_words"],
        output_dim=embedding_output_dim,
        input_length=params["data"]["max_sequence_length"]
    ))
    # Conv1D katmanları
    conv_layers = hp.Choice('conv_layers', params["model"]["conv1d"]["num_layers"])
    for i in range(conv_layers):
        filters = hp.Choice(f'filters_{i}', params["model"]["conv1d"]["filters"])
        kernel_size = hp.Choice(f'kernel_size_{i}', params["model"]["conv1d"]["kernel_sizes"])
        model.add(Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            activation='relu'
        ))
        model.add(MaxPooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    # Dense katmanları
    dense_layers = hp.Choice('dense_layers', params["model"]["dense"]["num_layers"])
    for i in range(dense_layers):
        units = hp.Choice(f'dense_units_{i}', params["model"]["dense"]["units"])
        activation = hp.Choice(f'dense_activation_{i}', params["model"]["dense"]["activations"])
        dropout_rate = hp.Choice(f'dropout_rate_{i}', params["model"]["dense"]["dropout_rates"])
        model.add(Dense(units=units, activation=activation))
        model.add(Dropout(rate=dropout_rate))
    # Çıkış katmanı
    model.add(Dense(1, activation='sigmoid'))
    # Optimizer seçimi
    optimizer_choice = hp.Choice('optimizer', params["model"]["optimizer"])
    learning_rate = hp.Choice('learning_rate', params["model"]["learning_rates"])
    if optimizer_choice == 'adam':
        optimizer = legacy_optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = legacy_optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = legacy_optimizers.SGD(learning_rate=learning_rate)
    # Batch size hiperparametresi
    batch_size = hp.Choice('batch_size', params["model"]["batch_sizes"])
    # Model derleme
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# =======================
# Ana Program
# =======================

if __name__ == "__main__":
    # Verileri yükleme
    train_texts, train_labels, test_texts, test_labels = load_data(params)

    # Veri yükleme sonrası kontroller
    print(f"Eğitim verisi sayısı: {len(train_texts)}")
    print(f"Eğitim etiketi sayısı: {len(train_labels)}")
    print(f"Test verisi sayısı: {len(test_texts)}")
    print(f"Test etiketi sayısı: {len(test_labels)}")

    if len(train_texts) == 0 or len(train_labels) == 0:
        raise ValueError("Eğitim verisi veya etiketleri boş. Lütfen veri yükleme adımlarını kontrol edin.")
    if len(test_texts) == 0 or len(test_labels) == 0:
        raise ValueError("Test verisi veya etiketleri boş. Lütfen veri yükleme adımlarını kontrol edin.")

    # Tokenizer ayarı ve eğitimi
    print("Tokenizer eğitiliyor...")
    # Tokenizer'ı tüm veri seti üzerinde eğitmek yerine daha küçük bir örneklem kullanıyoruz
    sample_size = min(10000, len(train_texts))
    sample_texts = random.sample(train_texts, sample_size)
    tokenizer = Tokenizer(num_words=params["data"]["num_words"], oov_token="<OOV>")
    tokenizer.fit_on_texts(sample_texts)

    # Bellek kullanımını azaltmak için asıl metinleri sil
    del sample_texts
    gc.collect()

    # Hiperparametre optimizasyonu için tuner oluşturma
    tuner = kt.BayesianOptimization(
        build_model,
        objective='val_accuracy',
        max_trials=params["training"]["max_trials"],
        directory='hyperparam_tuning',
        project_name='phishing_detection_bayes'
    )

    # Erken durdurma callback'i
    stop_early = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=params["training"]["patience"]
    )

    # Hiperparametre araması
    tuner.search(
        DataGenerator(train_texts, train_labels, tokenizer, params["data"]["max_sequence_length"], batch_size=32),
        epochs=params["training"]["max_epochs"],
        validation_data=DataGenerator(test_texts, test_labels, tokenizer, params["data"]["max_sequence_length"], batch_size=32, shuffle=False),
        callbacks=[stop_early]
    )

    # En iyi hiperparametreleri alma
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(f"""
    En iyi hiperparametreler:
    - Embedding çıkış boyutu: {best_hps.get('embedding_output_dim')}
    - Conv1D katman sayısı: {best_hps.get('conv_layers')}
    - Dense katman sayısı: {best_hps.get('dense_layers')}
    - Öğrenme oranı: {best_hps.get('learning_rate')}
    - Optimizer: {best_hps.get('optimizer')}
    - Batch size: {best_hps.get('batch_size')}
    """)

    # En iyi batch_size değerini al
    batch_size = best_hps.get('batch_size')

    # En iyi modelle eğitimi tekrar yapma
    model = tuner.hypermodel.build(best_hps)
    history = model.fit(
        DataGenerator(train_texts, train_labels, tokenizer, params["data"]["max_sequence_length"], batch_size=batch_size),
        epochs=params["training"]["max_epochs"],
        validation_data=DataGenerator(test_texts, test_labels, tokenizer, params["data"]["max_sequence_length"], batch_size=batch_size, shuffle=False),
        callbacks=[stop_early]
    )

    # Modelin performansını görselleştirme
    plt.figure(figsize=(12, 4))
    # Doğruluk
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
    plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
    plt.xlabel('Epoch')
    plt.ylabel('Doğruluk')
    plt.legend()
    # Kayıp
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Eğitim Kaybı')
    plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
    plt.xlabel('Epoch')
    plt.ylabel('Kayıp')
    plt.legend()
    plt.show()

    # Test seti üzerinde değerlendirme
    test_loss, test_accuracy = model.evaluate(
        DataGenerator(test_texts, test_labels, tokenizer, params["data"]["max_sequence_length"], batch_size=batch_size, shuffle=False)
    )
    print(f"Test Doğruluğu: {test_accuracy:.4f}")

    # Modeli kaydetme (isteğe bağlı)
    model.save('phishing_detection_model.keras')


Trial 100 Complete [01h 12m 56s]
val_accuracy: 0.8225917220115662

Best val_accuracy So Far: 0.857160747051239
Total elapsed time: 19d 04h 10m 18s

Search: Running Trial #101

Value             |Best Value So Far |Hyperparameter
32                |192               |embedding_output_dim
2                 |1                 |conv_layers
160               |256               |filters_0
3                 |5                 |kernel_size_0
2                 |1                 |dense_layers
224               |64                |dense_units_0
relu              |elu               |dense_activation_0
0.5               |0.5               |dropout_rate_0
adam              |adam              |optimizer
0.0001            |0.001             |learning_rate
32                |32                |filters_1
3                 |7                 |kernel_size_1
32                |96                |filters_2
3                 |5                 |kernel_size_2
256               |256               |dense_units

KeyboardInterrupt: 

In [3]:
# Tüm denemelerin sonuçlarını almak için
for trial in tuner.oracle.get_best_trials(num_trials=params["training"]["max_trials"]):
    print(f"Trial ID: {trial.trial_id}")
    print(f"Hyperparameters: {trial.hyperparameters.values}")
    print(f"Score: {trial.score}")
    print("-" * 30)


Trial ID: 086
Hyperparameters: {'embedding_output_dim': 192, 'conv_layers': 1, 'filters_0': 256, 'kernel_size_0': 5, 'dense_layers': 1, 'dense_units_0': 64, 'dense_activation_0': 'elu', 'dropout_rate_0': 0.5, 'optimizer': 'adam', 'learning_rate': 0.001, 'filters_1': 32, 'kernel_size_1': 7, 'filters_2': 96, 'kernel_size_2': 5, 'dense_units_1': 256, 'dense_activation_1': 'elu', 'dropout_rate_1': 0.3, 'batch_size': 32, 'dense_units_2': 128, 'dense_activation_2': 'tanh', 'dropout_rate_2': 0.5}
Score: 0.857160747051239
------------------------------
Trial ID: 036
Hyperparameters: {'embedding_output_dim': 160, 'conv_layers': 1, 'filters_0': 256, 'kernel_size_0': 3, 'dense_layers': 1, 'dense_units_0': 64, 'dense_activation_0': 'elu', 'dropout_rate_0': 0.5, 'optimizer': 'adam', 'learning_rate': 0.001, 'filters_1': 160, 'kernel_size_1': 5, 'filters_2': 256, 'kernel_size_2': 7, 'dense_units_1': 192, 'dense_activation_1': 'tanh', 'dropout_rate_1': 0.4, 'batch_size': 32, 'dense_units_2': 256, 'den

In [4]:
tuner.results_summary(num_trials=100)

Results summary
Results in hyperparam_tuning/phishing_detection_bayes
Showing 100 best trials
Objective(name="val_accuracy", direction="max")

Trial 086 summary
Hyperparameters:
embedding_output_dim: 192
conv_layers: 1
filters_0: 256
kernel_size_0: 5
dense_layers: 1
dense_units_0: 64
dense_activation_0: elu
dropout_rate_0: 0.5
optimizer: adam
learning_rate: 0.001
filters_1: 32
kernel_size_1: 7
filters_2: 96
kernel_size_2: 5
dense_units_1: 256
dense_activation_1: elu
dropout_rate_1: 0.3
batch_size: 32
dense_units_2: 128
dense_activation_2: tanh
dropout_rate_2: 0.5
Score: 0.857160747051239

Trial 036 summary
Hyperparameters:
embedding_output_dim: 160
conv_layers: 1
filters_0: 256
kernel_size_0: 3
dense_layers: 1
dense_units_0: 64
dense_activation_0: elu
dropout_rate_0: 0.5
optimizer: adam
learning_rate: 0.001
filters_1: 160
kernel_size_1: 5
filters_2: 256
kernel_size_2: 7
dense_units_1: 192
dense_activation_1: tanh
dropout_rate_1: 0.4
batch_size: 32
dense_units_2: 256
dense_activation_2:

In [11]:
# En iyi 5 modeli al
top_n = 5
best_models = tuner.get_best_hyperparameters(num_trials=top_n)

for i, hps in enumerate(best_models):
    print(f"Model {i+1}:")
    print(f"  - Embedding çıkış boyutu: {hps.get('embedding_output_dim')}")
    print(f"  - Conv1D katman sayısı: {hps.get('conv_layers')}")
    for j in range(hps.get('conv_layers')):
        print(f"    - Conv1D Layer {j+1} filtre sayısı: {hps.get(f'filters_{j}')}")
        print(f"    - Conv1D Layer {j+1} kernel boyutu: {hps.get(f'kernel_size_{j}')}")
    print(f"  - Dense katman sayısı: {hps.get('dense_layers')}")
    for j in range(hps.get('dense_layers')):
        print(f"    - Dense Layer {j+1} nöron sayısı: {hps.get(f'dense_units_{j}')}")
        print(f"  - Öğrenme oranı: {hps.get('learning_rate')}")
        print(f"  - Optimizer: {hps.get('optimizer')}")
        print(f"  - Batch size: {hps.get('batch_size')}")
    print("\n")

Model 1:
  - Embedding çıkış boyutu: 192
  - Conv1D katman sayısı: 1
    - Conv1D Layer 1 filtre sayısı: 256
    - Conv1D Layer 1 kernel boyutu: 5
  - Dense katman sayısı: 1
    - Dense Layer 1 nöron sayısı: 64
  - Öğrenme oranı: 0.001
  - Optimizer: adam
  - Batch size: 32


Model 2:
  - Embedding çıkış boyutu: 160
  - Conv1D katman sayısı: 1
    - Conv1D Layer 1 filtre sayısı: 256
    - Conv1D Layer 1 kernel boyutu: 3
  - Dense katman sayısı: 1
    - Dense Layer 1 nöron sayısı: 64
  - Öğrenme oranı: 0.001
  - Optimizer: adam
  - Batch size: 32


Model 3:
  - Embedding çıkış boyutu: 256
  - Conv1D katman sayısı: 1
    - Conv1D Layer 1 filtre sayısı: 96
    - Conv1D Layer 1 kernel boyutu: 7
  - Dense katman sayısı: 1
    - Dense Layer 1 nöron sayısı: 64
  - Öğrenme oranı: 0.001
  - Optimizer: adam
  - Batch size: 32


Model 4:
  - Embedding çıkış boyutu: 256
  - Conv1D katman sayısı: 1
    - Conv1D Layer 1 filtre sayısı: 224
    - Conv1D Layer 1 kernel boyutu: 5
  - Dense katman sayısı